In [ ]:
import pandas as pd
#from pandas.io.json import json_normalize
import json
from sqlalchemy import create_engine

In [ ]:
json_file = json.load(open('Resources/NYC_Tree_Census_2015.json'))
print(len(json_file["meta"]["view"]["columns"]))

column_names = []

for i in range(len(json_file["meta"]["view"]["columns"])):
    column_names.append(json_file["meta"]["view"]["columns"][i]["name"])
    
print(column_names)

tree_census_df = pd.DataFrame(json_file["data"], columns = column_names)
tree_census_df.head()


In [ ]:
tree_columns = ['tree_id', 'health', 'zipcode', 'boroname', 'address']
tree_transform = tree_census_df[tree_columns].copy()
tree_transform = tree_transform.rename(columns={'tree_id':'id',
                                                   'health':'tree_health',
                                                   'zipcode':'zipcode_id',
                                                   'boroname':'borough',
                                                   'address':'address'})
tree_transformed = tree_transform.dropna(how='any',axis=0)
tree_transformed.set_index('id', inplace=True)
tree_transformed.head()

In [ ]:
connection_string = "postgres:postgres@localhost:5432/tree_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
#next confirm the tables
engine.table_names()

In [ ]:
#next load the dataframe into SQL
tree_transformed.to_sql(name='tree_census', con=engine, if_exists='append', index=True)